In [3]:
import torch
import numpy as np
import pandas as pd
import time

from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as data_utils
from torch import flatten, tensor
import torch.nn as nn
import torch.nn.functional as F

In [4]:
df_input = pd.read_csv('input.csv', index_col=0, header=None)
df_output = pd.read_csv('output.csv', index_col=0, header=None)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/input.csv'

In [ ]:
INPUT_SIZE = 1200
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 256
EPOCHS = 1000
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def create_test_train_data(df_input, df_output):
    df_input = df_input.iloc[:,:INPUT_SIZE]
    df_output = df_output.iloc[:, -1]

    train_x = df_input.sample(frac=0.8)
    train_y = df_output.loc[train_x.index]

    test_x = df_input.drop(train_x.index)
    test_y = df_output.loc[test_x.index]

    num_train = len(train_x)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(VAL_SPLIT * num_train))
    train_index, valid_index = indices[split:], indices[:split]

    valid_x = train_x.iloc[valid_index, :]
    valid_y = train_y.iloc[valid_index]

    train_x = train_x.iloc[train_index, :]
    train_y = train_y.iloc[train_index]

    return train_x, train_y, valid_x, valid_y, test_x, test_y

In [ ]:
train_x, train_y, valid_x, valid_y, test_x, test_y = create_test_train_data(df_input, df_output)

In [ ]:
print(len(train_x), len(valid_x),len(test_x), len(df_input.index))

27994 9331 9331 46656


In [ ]:
train_target = tensor(train_y.values)
train = tensor(train_x.values) 
train_tensor = data_utils.TensorDataset(train, train_target) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)

valid_target = tensor(valid_y.values)
valid = tensor(valid_x.values) 
valid_tensor = data_utils.TensorDataset(valid, valid_target) 
valid_loader = data_utils.DataLoader(dataset = valid_tensor, batch_size = BATCH_SIZE, shuffle = True)

test_target = tensor(test_y.values)
test = tensor(test_x.values) 
test_tensor = data_utils.TensorDataset(test, test_target) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
class LeNet(nn.Module):

	def __init__(self):
		super(LeNet,self).__init__()
		# number of hidden nodes in each layer (512)
		hidden_1 = 512
		hidden_2 = 512
		# linear layer (784 -> hidden_1)
		self.fc1 = nn.Linear(INPUT_SIZE, 512)
		# linear layer (n_hidden -> hidden_2)
		self.fc2 = nn.Linear(512,512)
		# linear layer (n_hidden -> 10)
		self.fc3 = nn.Linear(512,1)
		self.fc4 = nn.Linear(INPUT_SIZE, INPUT_SIZE)
		self.act1 = F.relu

		# dropout layer (p=0.2)
		# dropout prevents overfitting of data
		self.droput = nn.Dropout(0.2)
    
	def forward(self, x):
		# add hidden layer, with relu activation function
		x = F.normalize(x)
		x = self.fc4(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc4(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc4(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc4(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc4(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc1(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		x = self.fc2(x)
		x = self.act1(x)
		x = self.droput(x)

		# add output layer
		x = self.fc3(x)
		x = flatten(x)
		return x

In [ ]:
print("[INFO] initializing the LeNet model...")
model = LeNet().to(device)
# initialize our optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001, weight_decay=1e-5)
criterion = nn.MSELoss()

[INFO] initializing the LeNet model...


In [ ]:
print("[INFO] training the network...")
startTime = time.time()
valid_loss_min = np.Inf
valid_error_min = np.Inf

for e in range(0, EPOCHS):
	# set the model in training mode
	# monitor losses
	train_loss = 0
	valid_loss = 0
	valid_error = 0
	
	###################
	# train the model #
	###################
	model.train() # prep model for training
	for data,label in train_loader:
		data, label = data.to(device), label.to(device)
		# clear the gradients of all optimized variables
		optimizer.zero_grad()
		# forward pass: compute predicted outputs by passing inputs to the model
		output = model(data.float())
		# calculate the loss
		#print(output,label.float() )
		loss = criterion(output,label.float())
		# backward pass: compute gradient of the loss with respect to model parameters
		loss.backward()
		# perform a single optimization step (parameter update)
		optimizer.step()
		# update running training loss
		train_loss += loss.item() * data.size(0)
		#print(train_loss, data.size(0), loss.item())
				
	######################    
	# validate the model #
	######################
	model.eval()  # prep model for evaluation
	for data,label in valid_loader:
		# forward pass: compute predicted outputs by passing inputs to the model
		data, label = data.to(device), label.to(device)
		output = model(data.float())
		# calculate the loss
		loss = criterion(output,label)
		# update running validation loss 
		valid_loss += loss.item() * data.size(0)
		valid_error += torch.sum(torch.div(torch.abs(output-label), torch.abs(label)))
	
	# print training/validation statistics 
	# calculate average loss over an epoch
	train_loss = train_loss / len(train_loader.sampler)
	valid_loss = valid_loss / len(valid_loader.sampler)
	valid_error = valid_error/ len(valid_loader.sampler)
	
	print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
		e+1, 
		train_loss,
		valid_loss
		))
	
	# save model if validation loss has decreased
	# if valid_loss <= valid_loss_min:
	# 	print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
	# 	valid_loss_min,
	# 	valid_loss))
	# 	torch.save(model.state_dict(), '/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/pytorch/models/perceptron.pt')
	# 	valid_loss_min = valid_loss

	if valid_error <= valid_error_min:
		print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
		valid_error_min,
		valid_error))
		torch.save(model.state_dict(), '/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/pytorch/models/perceptron.pt')
		valid_error_min = valid_error

print('Total time:', time.time()- startTime)

[INFO] training the network...
Epoch: 1 	Training Loss: 0.142784 	Validation Loss: 0.095856
Validation loss decreased (inf --> 6.858958).  Saving model ...
Epoch: 2 	Training Loss: 0.101262 	Validation Loss: 0.095724
Validation loss decreased (6.858958 --> 6.528288).  Saving model ...
Epoch: 3 	Training Loss: 0.096808 	Validation Loss: 0.094575
Validation loss decreased (6.528288 --> 6.275376).  Saving model ...
Epoch: 4 	Training Loss: 0.094681 	Validation Loss: 0.094571
Epoch: 5 	Training Loss: 0.094720 	Validation Loss: 0.088160
Validation loss decreased (6.275376 --> 6.082520).  Saving model ...
Epoch: 6 	Training Loss: 0.090662 	Validation Loss: 0.087427
Validation loss decreased (6.082520 --> 5.326997).  Saving model ...
Epoch: 7 	Training Loss: 0.087207 	Validation Loss: 0.085221
Epoch: 8 	Training Loss: 0.085462 	Validation Loss: 0.081533
Validation loss decreased (5.326997 --> 5.135009).  Saving model ...
Epoch: 9 	Training Loss: 0.085296 	Validation Loss: 0.081803
Epoch: 10 	

In [ ]:
# initialize lists to monitor test loss and accuracy
model.load_state_dict(torch.load('/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/pytorch/models/perceptron.pt'))
test_loss = 0.0
test_error = 0.0

model.eval() # prep model for evaluation
for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    data, target = data.to(device), target.to(device)
    output = model(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    test_error += torch.sum(torch.div(torch.abs(output-target), torch.abs(target)))

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
test_error = test_error/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))
print('Test Error: {:.6f}\n'.format(test_error))